In [0]:
pip install -r requirements.txt

# Import dependence

In [0]:
import os
import time
import math
import glob
import random
import argparse
import numpy as np
from PIL import Image
import scipy.stats as stats
import scipy.spatial as spatial

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

import dill
from robustness.datasets import ImageNet
from robustness.model_utils import make_and_restore_model

In [0]:
def convert_relu_to_softplus(model):
  for child_name, child in model.named_children():
      if isinstance(child, nn.ReLU):
          setattr(model, child_name, nn.Softplus())
      else:
          convert_relu_to_softplus(child)
  return model

# Hyper params

In [0]:
# the path of targeted model
model_path = None

# define top K
top_K = 1000

# max iter
max_iter = 50

# params for UG
num_steps = 50

# Load model and data

In [0]:
ds = ImageNet('/path/to/imagenet')

if model_path is None:
  model, _, = make_and_restore_model(arch='resnet50', dataset=ds,
              resume_path=None, pytorch_pretrained=True)
  softplus_model, _, = make_and_restore_model(arch='resnet50', dataset=ds,
              resume_path=None, pytorch_pretrained=True)
else:
  model, _, = make_and_restore_model(arch='resnet50', dataset=ds,
             resume_path=model_path, pytorch_pretrained=False)
  softplus_model, _, = make_and_restore_model(arch='resnet50', dataset=ds,
              resume_path=model_path, pytorch_pretrained=False)

vgg = model.eval().cuda()
vgg_softplus = convert_relu_to_softplus(softplus_model).eval().cuda()

In [0]:
data_path = 'data/'
image_path = 'data/images'
cls_label = open(os.path.join(data_path,'labels.txt'))
cls_label = cls_label.readlines()
print(len(cls_label))

# Attack and Evaluate

In [0]:
 def main(method = "topK", max_epsilon = 2.0):
  count = 0

  eps = max_epsilon / 255.0

  center_dislocation_sum = 0
  correlation_sum = 0
  intersection_sum = 0
  cosine_distance_sum = 0

  # generate a target map
  if method == "manipulate":
    line = cls_label[-1]
    line = line.strip('\n').split(' ')
    name = os.path.join(image_path,line[0])
    label = int(line[1])

    image = Image.open(name).convert('RGB')
    image = image.resize((224, 224), Image.ANTIALIAS)
    image = np.array(image) / 255.0

    # transfer Image into tensor, with shape NxCxHxW
    inputs = torch.from_numpy(image.transpose((2,0,1)))
    inputs = inputs.unsqueeze(0).float().requires_grad_(True).cuda()

    # get the saliency map using softplus model
    logit,_ = vgg_softplus(inputs)
    one_hot_output = torch.FloatTensor(1, logit.size()[-1]).zero_().cuda()
    one_hot_output[0][label] = 1
    target_map = torch.autograd.grad(torch.sum(logit[0]*one_hot_output), inputs, create_graph=True)[0] * inputs
    target_map = target_map.squeeze().detach()

    # combine color channel; normalized into (0,1) and scale by image size; flatten saliency map into 1D 
    normalized_target_map = torch.sum(torch.abs(target_map),0)
    normalized_target_map = 224*224*normalized_target_map/torch.sum(normalized_target_map)
    normalized_target_map = normalized_target_map.detach()

  # start the loop
  tic = time.time()
  for line in cls_label:
    count += 1

    line = line.strip('\n').split(' ')
    name = os.path.join(image_path,line[0])
    label = int(line[1])

    image = Image.open(name).convert('RGB')
    image = image.resize((224, 224), Image.ANTIALIAS)
    image = np.array(image) / 255.0

    '''
    original result on softplus model
    '''
    # transfer Image into tensor, with shape NxCxHxW
    inputs = torch.from_numpy(image.transpose((2,0,1)))
    inputs = inputs.unsqueeze(0).float().requires_grad_(True).cuda()

    # find intergated inputs
    sigma = 0.2 * (torch.max(inputs) - torch.min(inputs)).item()
    counterfactuals = [inputs+inputs.data.new(inputs.size()).uniform_(-sigma, sigma).cuda() for i in range(num_steps)]
    counterfactuals = torch.cat(counterfactuals)
    counterfactuals = counterfactuals.requires_grad_(True)

    # get the UG map using softplus model
    logit,_ = vgg_softplus(counterfactuals)
    original_logit = logit.clone()
    one_hot_output = torch.FloatTensor(1, logit.size()[-1]).zero_().cuda()
    one_hot_output[0][label] = 1
    original_saliency = torch.autograd.grad(torch.sum(logit*one_hot_output), counterfactuals)[0].detach()
    original_saliency = torch.mean(original_saliency, dim=0)
    original_saliency = original_saliency * inputs.squeeze()

    # combine color channel; normalized into (0,1) and scale by image size; flatten saliency map into 1D 
    normalized_original_saliency = torch.sum(torch.abs(original_saliency),0)
    normalized_original_saliency = 224*224*normalized_original_saliency/torch.sum(normalized_original_saliency)
    normalized_original_saliency_flatten = normalized_original_saliency.flatten()

    # get the mass center of original saliency map
    y_mesh, x_mesh = np.meshgrid(np.arange(224),np.arange(224))
    y_mesh, x_mesh = torch.Tensor(y_mesh).cuda(), torch.Tensor(x_mesh).cuda()
    mass_center = [int(torch.sum(normalized_original_saliency*x_mesh)/(224*224)), int(torch.sum(normalized_original_saliency*y_mesh)/(224*224))]


    '''
    attack
    '''
    adv_image = inputs.detach().clone()
    best_adv_image = adv_image.clone()
    max_mass_center_loss = 0
    for i in range(max_iter):
      
      adv_image = adv_image.clone().detach().requires_grad_(True)

      # find intergated inputs
      sigma = 0.2 / (torch.max(adv_image) - torch.min(adv_image)).item()
      counterfactuals = [adv_image+adv_image.data.new(adv_image.size()).uniform_(-sigma, sigma) for i in range(num_steps)]
      counterfactuals = torch.cat(counterfactuals)

      # get the SG map using softplus model
      logit,_ = vgg_softplus(counterfactuals)
      one_hot_output = torch.FloatTensor(1, logit.size()[-1]).zero_().cuda()
      one_hot_output[0][label] = 1
      if method == 'random':
        saliency = torch.autograd.grad(torch.sum(logit*one_hot_output), counterfactuals, create_graph=False)[0]
      else:
        saliency = torch.autograd.grad(torch.sum(logit*one_hot_output), counterfactuals, create_graph=True)[0]
      saliency = torch.mean(saliency, dim=0)
      saliency = saliency * adv_image.squeeze()

      # combine color channel; normalized into (0,1) and scale by image size; flatten saliency map into 1D 
      saliency = torch.sum(torch.abs(saliency),0)
      saliency = 224*224*saliency/torch.sum(saliency)
      saliency_flatten = saliency.flatten()

      # find the gradient direction
      if method == "topK":
        mask = torch.zeros(224*224).cuda()
        mask[torch.argsort(saliency.view(-1))[-top_K:]] = 1
        topK_loss = -torch.sum(saliency_flatten*mask)
        topK_direction = torch.autograd.grad(topK_loss, adv_image)[0]
        grad_sign = topK_direction.sign()
      elif method == "mass_center":
        mass_center_perturbed = [torch.sum(saliency*x_mesh)/(224*224), torch.sum(saliency*y_mesh)/(224*224)]
        mass_center_loss = ((mass_center[0]-mass_center_perturbed[0])**2 + (mass_center[1]-mass_center_perturbed[1])**2)
        mass_center_direction = torch.autograd.grad(mass_center_loss, adv_image)[0]
        grad_sign = mass_center_direction.sign()
      elif method == "manipulate":
        # if target_map is None:
        #   raise ValueError("target map is None!")
        # else:
        loss_expl = F.mse_loss(saliency, normalized_target_map)
        loss_output = F.mse_loss(original_logit, logit)
        total_loss = 1e11*loss_expl + 1e6*loss_output
        grad_sign = - torch.autograd.grad(total_loss, adv_image)[0].sign()
      elif method == "target":
        if target_map is None:
          raise ValueError("target map is None!")
        else:
          target_loss = torch.sum(saliency*target_map)
          grad_sign = torch.autograd.grad(target_loss, adv_image)[0].sign()
      else:
        grad_sign = torch.randn(adv_image.shape).cuda().sign()

      # apply perturbation
      adv_image = inputs + torch.clamp(adv_image+0.005*grad_sign-inputs, -eps, eps)
      adv_image = torch.clamp(adv_image, 0, 1)

      # evaluate each adv_image on given metric, save the best adv_image
      if vgg(inputs)[0].max(1)[-1] == vgg(adv_image)[0].max(1)[-1]:
          best_adv_image = adv_image.detach().clone()

    '''
    Evaluate on metrics
    '''
    # transfer Image into tensor, with shape NxCxHxW
    inputs = torch.from_numpy(image.transpose((2,0,1)))
    inputs = inputs.unsqueeze(0).float().requires_grad_(True).cuda()

    # find intergated inputs
    sigma = 0.2 / (torch.max(inputs) - torch.min(inputs)).item()
    counterfactuals = [inputs+inputs.data.new(inputs.size()).uniform_(-sigma, sigma).cuda() for i in range(num_steps)]
    counterfactuals = torch.cat(counterfactuals)
    counterfactuals = counterfactuals.requires_grad_(True)

    # get the UG map using softplus model
    logit,_ = vgg(counterfactuals)
    one_hot_output = torch.FloatTensor(1, logit.size()[-1]).zero_().cuda()
    one_hot_output[0][label] = 1
    original_saliency = torch.autograd.grad(torch.sum(logit*one_hot_output), counterfactuals)[0]
    original_saliency = torch.mean(original_saliency, dim=0)
    original_saliency = original_saliency * inputs.squeeze()

    # combine color channel; normalized into (0,1) and scale by image size; flatten saliency map into 1D 
    normalized_original_saliency = torch.sum(torch.abs(original_saliency),0)
    normalized_original_saliency = 224*224*normalized_original_saliency/torch.sum(normalized_original_saliency)
    normalized_original_saliency_flatten = normalized_original_saliency.flatten()

    # get the mass center of original saliency map
    y_mesh, x_mesh = np.meshgrid(np.arange(224),np.arange(224))
    y_mesh, x_mesh = torch.Tensor(y_mesh).cuda(), torch.Tensor(x_mesh).cuda()
    mass_center = [torch.sum(normalized_original_saliency*x_mesh)/(224*224), torch.sum(normalized_original_saliency*y_mesh)/(224*224)]

    # get perturbed input
    adv_image = best_adv_image.clone()

    sigma = 0.2 / (torch.max(adv_image) - torch.min(adv_image)).item()
    perturb_counterfactuals = [adv_image+adv_image.data.new(adv_image.size()).uniform_(-sigma, sigma).cuda() for i in range(num_steps)]
    perturb_counterfactuals = torch.cat(perturb_counterfactuals).requires_grad_(True)

    # get the perturbed saliency map using relu model
    logit,_ = vgg(perturb_counterfactuals)
    one_hot_output = torch.FloatTensor(1, logit.size()[-1]).zero_().cuda()
    one_hot_output[0][label] = 1
    perturb_saliency = torch.autograd.grad(torch.sum(logit*one_hot_output), perturb_counterfactuals)[0]
    perturb_saliency = torch.mean(perturb_saliency, dim=0)
    perturb_saliency = perturb_saliency * adv_image.squeeze()

    # combine color channel; normalized into (0,1) and scale by image size; flatten saliency map into 1D 
    normalized_perturb_saliency = torch.sum(torch.abs(perturb_saliency),0)
    normalized_perturb_saliency = 224*224*normalized_perturb_saliency/torch.sum(normalized_perturb_saliency)
    normalized_perturb_saliency_flatten = normalized_perturb_saliency.flatten()

    # get the mass center of perturbed saliency map
    mass_center_perturbed = [torch.sum(normalized_perturb_saliency*x_mesh)/(224*224), torch.sum(normalized_perturb_saliency*y_mesh)/(224*224)]

    center_dislocation = torch.sqrt(((mass_center[0]-mass_center_perturbed[0])**2 + (mass_center[1]-mass_center_perturbed[1])**2)).cpu().detach().numpy()
    correlation = stats.spearmanr(normalized_original_saliency_flatten.cpu().detach().numpy(), normalized_perturb_saliency_flatten.cpu().detach().numpy())

    top_val, top_idx = torch.topk(normalized_original_saliency_flatten, top_K)
    pert_val, pert_idx = torch.topk(normalized_perturb_saliency_flatten, top_K)
    intersection = float(len(np.intersect1d(top_idx.cpu().detach().numpy(), pert_idx.cpu().detach().numpy())))/top_K
    cosine_distance = float(spatial.distance.cosine(normalized_original_saliency_flatten.cpu().detach().numpy(), normalized_perturb_saliency_flatten.cpu().detach().numpy()))

    center_dislocation_sum += center_dislocation
    correlation_sum += correlation[0]
    intersection_sum += intersection
    cosine_distance_sum += cosine_distance

    torch.cuda.empty_cache()

  print("#######################")
  print('method:'+method+' '+'eps:'+str(max_epsilon))
  print("It spend {} to process {}/{}.".format(time.time()-tic, count, len(cls_label)))
  print('average center dislocation:', center_dislocation_sum / count)
  print('average correlation:', correlation_sum / count)
  print('average intersection:', intersection_sum / count)
  print('average cosine distance', cosine_distance_sum / count)

In [0]:
main(method = "topK", max_epsilon = 2.0)
main(method = "topK", max_epsilon = 4.0)
main(method = "topK", max_epsilon = 8.0)
main(method = "topK", max_epsilon = 16.0)

In [0]:
main(method = "manipulate", max_epsilon = 2.0)
main(method = "manipulate", max_epsilon = 4.0)
main(method = "manipulate", max_epsilon = 8.0)
main(method = "manipulate", max_epsilon = 16.0)

In [0]:
main(method = "mass_center", max_epsilon = 2.0)
main(method = "mass_center", max_epsilon = 4.0)
main(method = "mass_center", max_epsilon = 8.0)
main(method = "mass_center", max_epsilon = 16.0)

# Visualization

In [0]:
# for visual comparison
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams["figure.figsize"] = 8,8
plt.rc("text",usetex=False)
plt.rc("font",family = "sans-serif",size = 12)

'''
visualization
'''
# find intergated inputs
sigma = 0.2 / (torch.max(inputs) - torch.min(inputs)).item()
counterfactuals = [inputs+inputs.data.new(inputs.size()).uniform_(-sigma, sigma).cuda() for i in range(num_steps)]
counterfactuals = torch.cat(counterfactuals)
counterfactuals = counterfactuals.requires_grad_(True)

# get the UG map using relu model
logit,_ = vgg(counterfactuals)
one_hot_output = torch.FloatTensor(1, logit.size()[-1]).zero_().cuda()
one_hot_output[0][label] = 1
original_saliency = torch.autograd.grad(torch.sum(logit*one_hot_output), counterfactuals)[0]
original_saliency = torch.mean(original_saliency, dim=0)
original_saliency = original_saliency * inputs.squeeze()

saliency = np.sum(np.abs(original_saliency.detach().cpu().numpy()),0)
original_saliency = 224*224*saliency/np.sum(saliency)
plt.subplot(2,2,1)
plt.title("Original Image")
plt.imshow(inputs.squeeze().permute(1,2,0).detach().cpu().numpy())
plt.subplot(2,2,2)
plt.title("Original Image Saliency Map")
plt.imshow(original_saliency,cmap="hot")

# find intergated adv image
sigma = 0.2 / (torch.max(adv_image) - torch.min(adv_image)).item()
perturb_counterfactuals = [adv_image+adv_image.data.new(adv_image.size()).uniform_(-sigma, sigma).cuda() for i in range(num_steps)]
perturb_counterfactuals = torch.cat(perturb_counterfactuals).requires_grad_(True)

# get the IG map using relu model
logit,_ = vgg(perturb_counterfactuals)
one_hot_output = torch.FloatTensor(1, logit.size()[-1]).zero_().cuda()
one_hot_output[0][label] = 1
perturb_saliency = torch.autograd.grad(torch.sum(logit*one_hot_output), perturb_counterfactuals)[0]
perturb_saliency = torch.mean(perturb_saliency, dim=0)
perturb_saliency = perturb_saliency * adv_image.squeeze()

saliency = np.sum(np.abs(perturb_saliency.detach().cpu().numpy()),0)
perturbed_saliency = 224*224*saliency/np.sum(saliency)
plt.subplot(2,2,3)
plt.title("Perturbed Image")
plt.imshow(adv_image.squeeze().detach().permute(1,2,0).cpu().numpy())
plt.subplot(2,2,4)
plt.title("Perturbed Image Saliency Map")
plt.imshow(perturbed_saliency,cmap="hot")